In [3]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
import warnings; warnings.filterwarnings("ignore")

In [112]:
puntos_especies = pd.read_csv("..\\data\\observacion_puntos\\especies.csv")
cuadrantes_especie = pd.read_csv("..\\data\\observacion_puntos\\cuadrantes_especie.csv")

In [113]:
puntos_especies.head()

,latitud,longitud,especie,localidad
0,460902.306,3100672.715,Lotus kunkelii,Jinámar
1,460924.690,3100684.145,Lotus kunkelii,Jinámar
2,460932.548,3100657.633,Lotus kunkelii,Jinámar


In [114]:
cuadrantes_especie.head()

,latitud,longitud,especie,localidad,cuadrante
0,460888.97,3100675.65,Lotus kunkelii,Jinámar,Antes
1,460925.96,3100693.11,Lotus kunkelii,Jinámar,Antes
2,460932.94,3100682.95,Lotus kunkelii,Jinámar,Antes
3,460902.78,3100664.06,Lotus kunkelii,Jinámar,Antes
4,460888.97,3100675.65,Lotus kunkelii,Jinámar,Despues


In [125]:
## Separamos los datos
filtro_cuadrante1 = cuadrantes_especie["cuadrante"] == "Antes"
filtro_cuadrante2 = cuadrantes_especie["cuadrante"] == "Despues"

df_cuadrantes_antes = cuadrantes_especie[filtro_cuadrante1]
df_cuadrantes_despues = cuadrantes_especie[filtro_cuadrante2]

## Crear una capa SHP de puntos, marcando las especies encontradas

In [126]:
geometria_puntos = [Point(xy) for xy in zip(puntos_especies["latitud"], puntos_especies["longitud"])]
gdf_puntos_especie = gpd.GeoDataFrame(puntos_especies, geometry=geometria_puntos, crs="epsg:32628")
gdf_puntos_especie.head()

,latitud,longitud,especie,localidad,geometry
0,460902.306,3100672.715,Lotus kunkelii,Jinámar,POINT (460902.306 3100672.715)
1,460924.690,3100684.145,Lotus kunkelii,Jinámar,POINT (460924.690 3100684.145)
2,460932.548,3100657.633,Lotus kunkelii,Jinámar,POINT (460932.548 3100657.633)


## Crear una capa SHP con los polígonos para mostrar la extensión de tierra de las especies

Primero hay que crear una capa de puntos como el apartado anterior

In [127]:
geometria_cuadrantes_antes = [Point(xy) for xy in zip(df_cuadrantes_antes["latitud"], df_cuadrantes_antes["longitud"])]
gdf_puntos_cuadrantes_antes = gpd.GeoDataFrame(df_cuadrantes_antes, geometry=geometria_cuadrantes_antes, crs="epsg:32628")

geometria_cuadrantes_despues = [Point(xy) for xy in zip(df_cuadrantes_despues["latitud"], df_cuadrantes_despues["longitud"])]
gdf_puntos_cuadrantes_despues = gpd.GeoDataFrame(df_cuadrantes_despues, geometry=geometria_cuadrantes_despues, crs="epsg:32628")

In [128]:
gdf_puntos_cuadrantes_antes.head()

,latitud,longitud,especie,localidad,cuadrante,geometry
0,460888.97,3100675.65,Lotus kunkelii,Jinámar,Antes,POINT (460888.970 3100675.650)
1,460925.96,3100693.11,Lotus kunkelii,Jinámar,Antes,POINT (460925.960 3100693.110)
2,460932.94,3100682.95,Lotus kunkelii,Jinámar,Antes,POINT (460932.940 3100682.950)
3,460902.78,3100664.06,Lotus kunkelii,Jinámar,Antes,POINT (460902.780 3100664.060)


In [129]:
gdf_puntos_cuadrantes_despues

,latitud,longitud,especie,localidad,cuadrante,geometry
4,460888.97,3100675.65,Lotus kunkelii,Jinámar,Despues,POINT (460888.970 3100675.650)
5,460925.96,3100693.11,Lotus kunkelii,Jinámar,Despues,POINT (460925.960 3100693.110)
6,460943.90,3100660.09,Lotus kunkelii,Jinámar,Despues,POINT (460943.900 3100660.090)
7,460931.67,3100647.39,Lotus kunkelii,Jinámar,Despues,POINT (460931.670 3100647.390)


Pero esta vez además hay que unirlo para dar lugar a polígonos

In [130]:
poligono_cuadrante1 = Polygon(gdf_puntos_cuadrantes_antes["geometry"])
poligono_cuadrante2 = Polygon(gdf_puntos_cuadrantes_despues["geometry"])


In [131]:
## Creamos el data frame
gdf_cuadrantes_poligonos = gpd.GeoDataFrame({"cuadrante": cuadrantes_especie["cuadrante"].unique(), 
                                             "geometry": [poligono_cuadrante1, 
                                                          poligono_cuadrante2]})
print(gdf_cuadrantes_poligonos)

  cuadrante                                           geometry
0     Antes  POLYGON ((460888.970 3100675.650, 460925.960 3...
1   Despues  POLYGON ((460888.970 3100675.650, 460925.960 3...


## Descargamos las capas como archivos SHP

In [93]:
gdf_puntos_especie.to_file("..\\data\\gran_canaria\\puntos_especie.shp")


In [132]:
gdf_cuadrantes_poligonos.to_file("..\\data\\gran_canaria\\poligonos_especie.shp")